In [46]:
# from __future__ import division
import sys, os
BIN = os.path.expanduser("../../../")
sys.path.append(BIN)
import numpy as np
import matplotlib.pyplot as plt
from scipy.constants import c, e

In [ ]:
from PyHEADTAIL_feedback.signal_tools.signal_generators import SimpleBeam, SimpleCircularPointBeam
from PyHEADTAIL_feedback.signal_tools.trackers_and_kickers import track_beam, Wake
from PyHEADTAIL_feedback.signal_tools.plotters import plot_beam

In [ ]:


# SIMULATION PARAMETERS
# ---------------------
n_turns = 1000
n_buckets_per_bunch = 1

# MACHINE PARAMETERS
#-------------------
circumference = 9770.14
p0 = 3300e9 * e / c
accQ_x = 109.28
beta_x = circumference / (2.*np.pi*accQ_x)
n_bunches=1306
bunch_spacing=25e-9
intensity=1e13


# GENERATES BEAM
# --------------
beam = SimpleCircularPointBeam(n_bunches, bunch_spacing, intensity, circular_overlapping=0)

# beam = SimpleBeam(n_bunches, bunch_spacing, bunch_length, intensity, n_slices, beta_x=beta_x)
beam.set_beam_paramters(p0)
beam.init_noise(1e-7)


# GENERATES WAKES
# --------------
wakedata = np.loadtxt('example_impedance.wake')
print 'wakedata.shape' + str(wakedata.shape)
data_t = wakedata[:,0]
data_x = wakedata[:,2]
wake = Wake(data_t,data_x,10)


trackers = [
    wake
]
fig, ax1, ax11 = plot_beam(beam)

track_beam(beam,trackers,n_turns,accQ_x)
print 'Done'

fig, ax1, ax11 = plot_beam(beam,fig=fig, ax1=ax1, ax11=ax11)
plt.show()

9788.2237537
{}
wakedata.shape(11822, 6)


In [ ]:
print beam.x
print np.max(beam.x)